In [11]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.3 pyspark-shell'
#os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-1.8.0-openjdk-1.8.0.212.b04-0.el7_6.x86_64/jre/bin/java'
#spark.stop()
import sys
import time
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils


topic = "test3"  #should come from Mappings tbl
streamWindowTime=60 #should come from Mappings tbl
streamWaitTime=90 #should come from Mappings tbl
rawZoneFileName="hdfs://10.3.2.13:8020/user/hadoop/rawzone/kafka" #should come from Mappings tbl
bootstrapServers='10.3.2.25:9092' #should come from Mappings tbl

conf = SparkConf().setAppName("Kafka-spark-Hdfs").setMaster("local[*]")
sc = SparkContext(conf = conf)
#sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
ssc = StreamingContext(sc,streamWindowTime)

kafkaStream = KafkaUtils.createDirectStream(ssc,[topic],{'bootstrap.servers':bootstrapServers,'group.id':'video-group','fetch.message.max.bytes':'15728640','auto.offset.reset':'largest'})

stream = kafkaStream.map(lambda x: x[1])
stream.pprint()
stream.saveAsTextFiles(rawZoneFileName)

ssc.start()
time.sleep(streamWaitTime)
ssc.stop()

Exception: Java gateway process exited before sending its port number